In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd
%cd '/content/drive/My Drive/Colab Notebooks/cs505/'
!pwd

/content
/content/drive/My Drive/Colab Notebooks/cs505
/content/drive/My Drive/Colab Notebooks/cs505


In [ ]:
pip install tune-sklearn scikit-optimize ray[tune] 

     |████████████████████████████████| 40 kB 3.3 MB/s 
     |████████████████████████████████| 100 kB 4.3 MB/s 
     |████████████████████████████████| 57.6 MB 2.1 kB/s 
     |████████████████████████████████| 119 kB 60.6 MB/s 
     |████████████████████████████████| 124 kB 51.4 MB/s 


In [ ]:
from tune_sklearn import TuneSearchCV

In [ ]:
import csv
import os
import pandas as pd
import numpy as np

import lightgbm as lgb
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt

from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import sklearn.metrics as metrics
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer

In [ ]:
train = pd.read_csv('./train.csv', usecols=[ 'imdb_id', 'budget', 'popularity','runtime', 'release_date', 'revenue'])

train = pd.merge(train, pd.read_csv('./TrainAdditionalFeatures.csv'), how='left', on=['imdb_id'])

In [ ]:
train1 = train.drop(train[train['budget'] < 1000].index)

train2 = train1.drop(train1[train1['runtime'] < 60].index)

train2[['release_month','release_day','release_year']]=train2['release_date'].str.split('/',expand=True).replace(np.nan, 0).astype(int)
train2['release_year'] = train2['release_year']
train2.loc[ (train2['release_year'] <= 18) & (train2['release_year'] < 100), "release_year"] += 2000
train2.loc[ (train2['release_year'] > 18)  & (train2['release_year'] < 100), "release_year"] += 1900

train2 = train2.dropna()


result = train2.revenue

trainfeature = train2.drop(labels="revenue", axis=1)
trainfeature1 = trainfeature.drop(labels="release_date",axis=1)
trainfeature2 = trainfeature1.drop(labels="imdb_id",axis=1)


X_train, X_test, y_train, y_test = train_test_split(
    trainfeature2, result, test_size=0.20)


#creating Scoring parameter: 
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),'recall':make_scorer(recall_score)}

In [ ]:
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01],
    "min_samples_split": [3, 5, 7],
    "min_samples_leaf": [3, 5, 7],
    "max_depth":[3,5,8],
    "n_estimators":[50]
    }
#passing the scoring function in the GridSearchCV
clf = TuneSearchCV(GradientBoostingClassifier(),parameters,cv=3,scoring = 'r2',verbose = 2,return_train_score=True)

clf.fit(X_train, y_train)
#converting the clf.cv_results to dataframe
df=pd.DataFrame.from_dict(clf.cv_results_)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
clf.best_estimator_

GradientBoostingClassifier(learning_rate=0.01, min_samples_leaf=5,
                           min_samples_split=7, n_estimators=50)

In [ ]:
clf.best_score_

0.2914848868431057

In [ ]:
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01],
    "min_samples_split": [7],
    "min_samples_leaf": [5],
    "max_depth":[5,8],
    "n_estimators":[100]
    }
#passing the scoring function in the GridSearchCV
clf2 = TuneSearchCV(GradientBoostingClassifier(),parameters,cv=3,scoring = 'r2',verbose = 2,return_train_score=True)

clf2.fit(X_train, y_train)
#converting the clf.cv_results to dataframe
df2=pd.DataFrame.from_dict(clf2.cv_results_)
print(clf.best_estimator_)
print(clf.best_score_)

/usr/local/lib/python3.7/dist-packages/ray/tune/tune.py:374: UserWarning: The `loggers` argument is deprecated. Please pass the respective `LoggerCallback` classes to the `callbacks` argument instead. See https://docs.ray.io/en/latest/tune/api_docs/logging.html
  "The `loggers` argument is deprecated. Please pass the respective "


(_Trainable pid=789) /usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
(_Trainable pid=789)   UserWarning,
(_Trainable pid=791) /usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
(_Trainable pid=791)   UserWarning,


Trial _Trainable_ddedd_00001 reported average_test_score=0.25 with parameters={'early_stopping': False, 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'X_id': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000004000000), 'y_id': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000005000000), 'groups': None, 'cv': StratifiedKFold(n_splits=3, random_state=None, shuffle=False), 'fit_params': {}, 'scoring': {'score': make_scorer(r2_score)}, 'max_iters': 1, 'return_train_score': True, 'n_jobs': 1, 'metric_name': 'average_test_score', 'loss': 'deviance', 'learning_rate': 0.01, 'min_samples_split': 7, 'min_samples_leaf': 5, 'max_depth': 5, 'n_estimators': 100, 'estimator_ids': [ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000006000000)]}. This trial completed.


Trial _Trainable_ddedd_00000 reported average_test_score=0.23 with parameters={'early_stopping': False, 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'X_id': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000004000000), 'y_id': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000005000000), 'groups': None, 'cv': StratifiedKFold(n_splits=3, random_state=None, shuffle=False), 'fit_params': {}, 'scoring': {'score': make_scorer(r2_score)}, 'max_iters': 1, 'return_train_score': True, 'n_jobs': 1, 'metric_name': 'average_test_score', 'loss': 'deviance', 'learning_rate': 0.01, 'min_samples_split': 7, 'min_samples_leaf': 5, 'max_depth': 8, 'n_estimators': 100, 'estimator_ids': [ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000006000000)]}. This trial completed.


Trial name,status,loc,learning_rate,loss,max_depth,min_samples_leaf,min_samples_split,n_estimators,iter,total time (s),split0_test_score,split1_test_score,split2_test_score
_Trainable_ddedd_00000,TERMINATED,172.28.0.2:791,0.01,deviance,8,5,7,100,1,12505.2,0.106998,0.233893,0.360204
_Trainable_ddedd_00001,TERMINATED,172.28.0.2:789,0.01,deviance,5,5,7,100,1,11634.9,0.100288,0.319005,0.3243


In [11]:
print(clf2.best_estimator_)
print(clf2.best_score_)

GradientBoostingClassifier(learning_rate=0.01, max_depth=5, min_samples_leaf=5,
                           min_samples_split=7)
0.2478642991323061


In [12]:
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.05],
    "min_samples_split": [5,8],
    "min_samples_leaf": [5,8],
    "max_depth":[5,8],
    "n_estimators":[50]
    }
#passing the scoring function in the GridSearchCV
clf3 = TuneSearchCV(GradientBoostingClassifier(),parameters,cv=3,scoring = 'r2',verbose = 2,return_train_score=True)

clf3.fit(X_train, y_train)
#converting the clf.cv_results to dataframe
df2=pd.DataFrame.from_dict(clf3.cv_results_)
print(clf3.best_estimator_)
print(clf3.best_score_)

Output hidden; open in https://colab.research.google.com to view.

In [14]:
print(clf3.best_estimator_)
print(clf3.best_score_)

GradientBoostingClassifier(learning_rate=0.05, max_depth=8, min_samples_leaf=5,
                           min_samples_split=5, n_estimators=50)
0.2414633012776528
